In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("data_cleaning_full.csv")
df.head()

,ID,Gender,Age,City,Education,Experience_Years,Salary
0,1,Male,35.0,Berlin,Bachelors,2.0,70000.0
1,2,Female,NaN,New York,Masters,4.0,50000.0
2,3,Male,28.0,NaN,Bachelors,1.0,30000.0
3,4,Male,40.0,New York,Masters,4.0,NaN
4,5,Male,35.0,New York,NaN,7.0,40000.0


# Then We simply check data stats as and see data type

In [5]:
df.shape
df.info()
df.describe(include='all').T


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                15 non-null     int64  
 1   Gender            15 non-null     object 
 2   Age               11 non-null     float64
 3   City              10 non-null     object 
 4   Education         12 non-null     object 
 5   Experience_Years  14 non-null     float64
 6   Salary            11 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 972.0+ bytes


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,15.0,NaN,NaN,NaN,8.0,4.472136,1.0,4.5,8.0,11.5,15.0
Gender,15,2,Male,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,11.0,NaN,NaN,NaN,31.818182,6.940003,22.0,25.0,35.0,37.5,40.0
City,10,3,Delhi,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education,12,3,Masters,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Experience_Years,14.0,NaN,NaN,NaN,3.071429,1.63915,1.0,2.0,3.0,4.0,7.0
Salary,11.0,NaN,NaN,NaN,45454.545455,12933.395814,30000.0,40000.0,40000.0,45000.0,70000.0


# Getting the missing data in percent for each column

In [6]:
missing = df.isnull().sum()
missing_percent = (missing / len(df) * 100).round(2)
pd.concat([missing, missing_percent.rename("percent")], axis=1)


,0,percent
ID,0,0.00
Gender,0,0.00
Age,4,26.67
City,5,33.33
Education,3,20.00
Experience_Years,1,6.67
Salary,4,26.67


In [7]:
dupes = df.duplicated().sum()
dupes, df[ df.duplicated() ].head()


(np.int64(0),
 Empty DataFrame
 Columns: [ID, Gender, Age, City, Education, Experience_Years, Salary]
 Index: [])

In [8]:
for col in ['Gender', 'City', 'Education']:
    df[col] = df[col].astype(str).str.strip().replace({'nan': np.nan})
    # If you want title case:
    df[col] = df[col].where(df[col].isnull(), df[col].str.title())


In [9]:
num_cols = df.select_dtypes(include=['number']).columns.tolist()
# Example: numeric imputation with median (robust to outliers)
df[num_cols] = df[num_cols].fillna(df[num_cols].median())


In [10]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
# Fill with mode or a placeholder
for c in cat_cols:
    df[c] = df[c].fillna(df[c].mode().iloc[0] if not df[c].mode().empty else "Unknown")


In [11]:
from sklearn.impute import SimpleImputer

num_imp = SimpleImputer(strategy='median')
cat_imp = SimpleImputer(strategy='most_frequent')

df[num_cols] = num_imp.fit_transform(df[num_cols])
df[cat_cols] = cat_imp.fit_transform(df[cat_cols])


In [12]:
# Example: using IQR for 'Salary'
col = 'Salary'
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
outliers = df[(df[col] < lower) | (df[col] > upper)]
outliers.shape
# Option 1: cap values
df[col] = df[col].clip(lower, upper)


# Now we do label encoding and hot encoding so ml model can understand the data

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Gender_enc'] = le.fit_transform(df['Gender'])
df = pd.get_dummies(df, columns=['City', 'Education'], drop_first=True)



KeyError: "None of [Index(['City', 'Education'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# choose numeric columns after imputation
num_cols = df.select_dtypes(include=['number']).columns.tolist()
df[num_cols] = scaler.fit_transform(df[num_cols])


In [ ]:
# Drop or keep:
df = df.drop(columns=['ID'])  # if 'ID' is just identifier
# Create new features if useful (example)
# df['Age_Squared'] = df['Age'] ** 2


In [20]:
df.info()
df.isnull().sum()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Gender                 15 non-null     object 
 1   Age                    15 non-null     float64
 2   Experience_Years       15 non-null     float64
 3   Salary                 15 non-null     float64
 4   Gender_enc             15 non-null     int64  
 5   City_Delhi             15 non-null     bool   
 6   City_New York          15 non-null     bool   
 7   Education_High School  15 non-null     bool   
 8   Education_Masters      15 non-null     bool   
dtypes: bool(4), float64(3), int64(1), object(1)
memory usage: 792.0+ bytes


,Gender,Age,Experience_Years,Salary,Gender_enc,City_Delhi,City_New York,Education_High School,Education_Masters
0,Male,0.399641,-0.698963,0.0,1,False,False,False,False
1,Female,0.399641,0.611593,0.0,0,False,True,False,True
2,Male,-0.799283,-1.354241,0.0,1,True,False,False,False
3,Male,1.256016,0.611593,0.0,1,False,True,False,True
4,Male,0.399641,2.577427,0.0,1,False,True,False,True


# Now we split data before training to check our model accuracy.

In [21]:
from sklearn.model_selection import train_test_split

# Suppose 'Salary' is target (if not, replace with your target)
X = df.drop(columns=['Salary'])
y = df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
df.to_csv("data_cleaning_full_cleaned.csv", index=False)


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

num_features = ['Age','Experience_Years']  # example numeric list
cat_features = ['Gender','City','Education']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

model_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit
# model_pipeline.fit(X_train, y_train)
